In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Intro\n",
    "\n",
    "This notebook explores the model we made to determine Titanic survival/death probability based on a number of variables."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Libraries\n",
    "\n",
    "The libraries are used for various tasks such as web development (Flask), data manipulation (pandas), visualization (seaborn), machine learning (scikit-learn), and numerical computations (numpy)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from flask import Flask, request, jsonify, Blueprint\n",
    "from flask_restful import Api, Resource\n",
    "import seaborn as sns\n",
    "import pandas as pd\n",
    "from sklearn.preprocessing import OneHotEncoder\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "import numpy as np"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Blueprint/Resoucre Creation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "titanic_api = Blueprint('titanic_api', __name__, url_prefix='/api/titanic')\n",
    "api = Api(titanic_api)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# TitanicAPI Class\n",
    "\n",
    "### Initialization\n",
    "\n",
    "--> Initialize machine learning model and prepare the Titanic dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "ename": "NameError",
     "evalue": "name 'Resource' is not defined",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mNameError\u001b[0m                                 Traceback (most recent call last)",
      "Cell \u001b[0;32mIn[2], line 1\u001b[0m\n\u001b[0;32m----> 1\u001b[0m \u001b[39mclass\u001b[39;00m \u001b[39mTitanicAPI\u001b[39;00m(Resource):\n\u001b[1;32m      2\u001b[0m     \u001b[39mdef\u001b[39;00m \u001b[39m__init__\u001b[39m(\u001b[39mself\u001b[39m):\n\u001b[1;32m      3\u001b[0m         \u001b[39m# Load the Titanic dataset\u001b[39;00m\n\u001b[1;32m      4\u001b[0m         titanic_data \u001b[39m=\u001b[39m sns\u001b[39m.\u001b[39mload_dataset(\u001b[39m'\u001b[39m\u001b[39mtitanic\u001b[39m\u001b[39m'\u001b[39m)\n",
      "\u001b[0;31mNameError\u001b[0m: name 'Resource' is not defined"
     ]
    }
   ],
   "source": [
    "class TitanicAPI(Resource):\n",
    "    def __init__(self):\n",
    "        # Load the Titanic dataset\n",
    "        titanic_data = sns.load_dataset('titanic')\n",
    "        \n",
    "        # Preprocess the dataset\n",
    "        td = titanic_data.copy()\n",
    "        td.drop(['alive', 'who', 'adult_male', 'class', 'embark_town', 'deck'], axis=1, inplace=True)\n",
    "        td.dropna(inplace=True)\n",
    "        td['sex'] = td['sex'].apply(lambda x: 1 if x == 'male' else 0)\n",
    "        td['alone'] = td['alone'].apply(lambda x: 1 if x else 0)\n",
    "\n",
    "        # Encode categorical variables\n",
    "        self.enc = OneHotEncoder(handle_unknown='ignore')\n",
    "        embarked_encoded = self.enc.fit_transform(td[['embarked']].values.reshape(-1, 1))\n",
    "        self.encoded_cols = self.enc.get_feature_names_out(['embarked'])\n",
    "        td[self.encoded_cols] = embarked_encoded.toarray()\n",
    "        td.drop(['embarked'], axis=1, inplace=True)\n",
    "\n",
    "        # Initialize Logistic Regression model\n",
    "        self.logreg = LogisticRegression(max_iter=1000)\n",
    "        X = td.drop('survived', axis=1)\n",
    "        y = td['survived']\n",
    "        self.logreg.fit(X, y)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### predict_survival Method\n",
    "\n",
    "Takes passenger data as input, preprocesses it --> uses the trained Logistic Regression model to predict survival probabilities."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "    def predict_survival(self, data):\n",
    "        try:\n",
    "            # Preprocess input data\n",
    "            passenger = pd.DataFrame([data]) \n",
    "            passenger['sex'] = passenger['sex'].apply(lambda x: 1 if x == 'male' else 0)\n",
    "            passenger['alone'] = passenger['alone'].apply(lambda x: 1 if x else 0)\n",
    "            embarked_encoded = self.enc.transform(passenger[['embarked']].values.reshape(-1, 1))\n",
    "            passenger[self.encoded_cols] = embarked_encoded.toarray()\n",
    "            passenger.drop(['embarked', 'name'], axis=1, inplace=True)\n",
    "\n",
    "            # Predict survival probabilities\n",
    "            dead_proba, alive_proba = np.squeeze(self.logreg.predict_proba(passenger))\n",
    "\n",
    "            return {\n",
    "                'Death probability': '{:.2%}'.format(dead_proba),\n",
    "                'Survival probability': '{:.2%}'.format(alive_proba)\n",
    "            }\n",
    "        except Exception as e:\n",
    "            return {'error': str(e)}"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Endpoint\n",
    "\n",
    "- Define a POST endpoint /predict to receive passenger data --> return survival predictions.\n",
    "- Endpoint essentially also handles exceptions and returns appropriate responses."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "ename": "NameError",
     "evalue": "name 'api' is not defined",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mNameError\u001b[0m                                 Traceback (most recent call last)",
      "Cell \u001b[0;32mIn[1], line 9\u001b[0m\n\u001b[1;32m      6\u001b[0m     \u001b[39mexcept\u001b[39;00m \u001b[39mException\u001b[39;00m \u001b[39mas\u001b[39;00m e:\n\u001b[1;32m      7\u001b[0m         \u001b[39mreturn\u001b[39;00m jsonify({\u001b[39m'\u001b[39m\u001b[39merror\u001b[39m\u001b[39m'\u001b[39m: \u001b[39mstr\u001b[39m(e)})\n\u001b[0;32m----> 9\u001b[0m api\u001b[39m.\u001b[39madd_resource(TitanicAPI, \u001b[39m'\u001b[39m\u001b[39m/predict\u001b[39m\u001b[39m'\u001b[39m)\n",
      "\u001b[0;31mNameError\u001b[0m: name 'api' is not defined"
     ]
    }
   ],
   "source": [
    "    def post(self):\n",
    "        try:\n",
    "            data = request.json\n",
    "            result = self.predict_survival(data)\n",
    "            return jsonify(result)\n",
    "        except Exception as e:\n",
    "            return jsonify({'error': str(e)})\n",
    "\n",
    "api.add_resource(TitanicAPI, '/predict')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.7"
  },
  "orig_nbformat": 4
 },
 "nbformat": 4,
 "nbformat_minor": 2
}